In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt


In [2]:
%matplotlib inline


In [3]:
%load_ext google.cloud.bigquery


In [14]:
%%bigquery status_count 

SELECT date_hour, 
       result,
       count, 
       cast(percentiles[offset(75)] as FLOAT64) p75_duration_ms,
       cast(percentiles[offset(95)] as FLOAT64) p95_duration_ms
  FROM (
    SELECT TIMESTAMP_TRUNC(a.timestamp, HOUR) as date_hour, 
          case  
            when status < 500 then 'success'
            else 'failed'
          end result, 
          count(*) as count, 
          APPROX_QUANTILES(cast(duration_ms as FLOAT64), 100) percentiles
    FROM gcp_perf_analysis.workhorse_thanksgiving a
    WHERE DATE(a.timestamp) = "2019-11-28" 
      AND status IS NOT NULL
      AND status > 0
      AND hostname LIKE 'web-%'
      AND uri NOT IN ('/-/readiness', '/-/liveness')
    GROUP BY 1,2 
    ORDER BY 1,2
  )



In [32]:
alt.Chart(status_count).mark_line().encode(
    y='count', 

    x='date_hour:T', 
    color='result',
    tooltip=['result:N'],
)


<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [34]:
%%bigquery web_request_durations 

SELECT date_hour, 
       count, 
       cast(percentiles[offset(75)] as FLOAT64) p75_duration_ms ,
       cast(percentiles[offset(95)] as FLOAT64) p95_duration_ms
  FROM (
    SELECT TIMESTAMP_TRUNC(a.timestamp, HOUR) as date_hour, 
          case  
            when status < 500 then 'success'
            else 'failed'
          end result, 
          count(*) as count, 
          APPROX_QUANTILES(cast(duration_ms as FLOAT64), 100) percentiles
    FROM gcp_perf_analysis.workhorse_thanksgiving a
    WHERE status IS NOT NULL
      AND status > 0
      AND status < 500
      AND hostname LIKE 'web-%'
      AND uri NOT IN ('/-/readiness', '/-/liveness')
    GROUP BY 1,2 
    ORDER BY 1,2
  )


In [41]:
alt.Chart(web_request_durations).mark_line().encode(
    y=alt.Y('p95_duration_ms', scale=alt.Scale(domain=[1, 20000], clamp=True)),
    x='date_hour:T',  
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
